# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Create list of filepaths to original event CSV data files for processing

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(len(file_path_list))

/home/workspace
30


#### Process the files and consolidate data in a single CSV file that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
               
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
#print(full_data_rows_list)

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Modeling Data with Apache Cassandra

## The above code created a CSV file titled <font color=red>event_datafile_new.csv</font> which contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Now we set up our Apache Cassandra cluster and keyspace

#### Create a cluster and connect to it

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

session = cluster.connect()

#### Create keyspace

In [6]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity 
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set keyspace for the current session

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## In each of the sections below, model a table for the given requirement, load data into the table, then execute a query to verify the success of the data load

### 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

##### In this table, we include the fields to be selected (`artist`, `song`, `length`) as well as the fields to filter on (`sessionId`, `itemInSession`). `sessionId` is set as partition key since listening session IDs are presumably evenly-distributed, and `itemInSession` is set as a clustering column since it provides additional information about the `sessionId`.

In [8]:
query = "CREATE TABLE IF NOT EXISTS songplays_by_session "
query += "(session_id int, item_in_session int, artist text, length decimal, song text, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songplays_by_session (session_id, item_in_session, artist, length, song)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))

In [10]:
query = "select artist, song, length from songplays_by_session WHERE session_id=338 AND item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userId = 10, sessionId = 182

##### In this table, we include the fields to be selected (`artist`, `song`, `firstName`, `lastName`), the fields to filter by (`userId`, `sessionId`), and the field to sort by (`itemInSession`). (`userId`, `sessionId`) is used as a composite partition key since `userId` uniquely identifies users, and because both fields are used to filter but not sort rows. Also, both `userId` and `sessionId` are presumably evenly-distributed across their range. `itemInSession` is set as the only clustering column since it is used to sort `song`, the only unique column to be selected.

In [11]:
query = "CREATE TABLE IF NOT EXISTS songplays_by_user_and_session "
query += "(user_id int, session_id int, item_in_session int, artist text, first_name text, last_name text, song text, PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songplays_by_user_and_session (user_id, session_id, item_in_session, artist, first_name, last_name, song)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]))

In [13]:
query = "select artist, song, first_name, last_name from songplays_by_user_and_session WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

##### In this table, we include the fields to be selected (`firstName`, `lastName`), the field to filter by (`song`) as partition key, and a third column (`userId`) to use as a clustering column to ensure there is a unique entry for each user that played a song and not only one entry per song.

In [14]:
query = "CREATE TABLE IF NOT EXISTS users_who_played_song_x "
query += "(song text, user_id int, first_name text, last_name text, PRIMARY KEY (song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_who_played_song_x (song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
query = "select first_name, last_name from users_who_played_song_x WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
query = "DROP TABLE IF EXISTS udacity.songplays_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS udacity.songplays_by_user_and_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS udacity.users_who_played_song_x"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()